In [57]:
import os
import pandas as pd
import json
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

columns = [
    'Min Temp',
    'Max Temp',
    'Total Precipitation',
    #'Max Gust',
    'Min Humidity',
    'Max Humidity',
    'Min Wind Speed',
    'Max Wind Speed',
    'Min Visibility',
    'Max Visibility',
    'Min Pressure',
    'Max Pressure',
]

def get_df_value(df, date, column):
    series = df[df['ds'] == date][column]
    return series.head(1).item()

def predict(dates):
    data = {}
    
    for column in columns:
        # Sanitize the column name to create a model name
        model_name = column.replace(' ', '_').lower()
        
        with open(os.path.join('..', 'Models', model_name + '.json'), 'r') as f:
            prophet = model_from_json(f.read())
        
        future_df = pd.DataFrame(dates, columns=[ 'ds' ])
        
        data[column] = prophet.predict(future_df)
    
    predict_rows = []
    
    for date in dates:
        predict_row = [ date, int(date[5:7]), int(date[8:10]) ]
        
        for column in columns:
            predict_row.append(get_df_value(data[column], date, 'yhat'))
        
        predict_rows.append(predict_row)
    
    predict_columns = [ 'ds', 'Month', 'Day' ]
    predict_columns.extend(columns)
    
    return pd.DataFrame(predict_rows, columns=predict_columns)

p_dates = [ '2023-03-31', '2023-04-01' ]
predict_df = predict(p_dates)

# Load the condition model
with open(os.path.join('..', 'Models', 'condition.pkl'), 'rb') as f:
    classifier = pickle.load(f)

classifier.predict(predict_df.drop('ds', axis=1))

for date in p_dates:
    print(date)
    print("----------")
    
    print("Temperature: {:.2f}°C to {:.2f}°C".format(get_df_value(predict_df, date, 'Min Temp'), get_df_value(predict_df, date, 'Max Temp')))
    print("Humidity: {:.2f}% to {:.2f}%".format(get_df_value(predict_df, date, 'Min Humidity'), get_df_value(predict_df, date, 'Max Humidity')))
    print("Wind Speed: {:.2f} km/h to {:.2f} km/h".format(get_df_value(predict_df, date, 'Min Wind Speed'), get_df_value(predict_df, date, 'Max Wind Speed')))
    print("Visibility: {:.2f} km to {:.2f} km".format(get_df_value(predict_df, date, 'Min Visibility'), get_df_value(predict_df, date, 'Max Visibility')))
    print("Pressure: {:.2f} kPa to {:.2f} kPa".format(get_df_value(predict_df, date, 'Min Pressure'), get_df_value(predict_df, date, 'Max Pressure')))
    
    print()

/usr/local/lib/python3.9/site-packages/prophet/serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))
/usr/local/lib/python3.9/site-packages/prophet/serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))
/usr/local/lib/python3.9/site-packages/prophet/serialize.py:160: FutureWar

ValueError: X has 13 features, but GradientBoostingClassifier is expecting 12 features as input.